## Introduction
This notebook is intended as a guide to my current toy project of making a
reinforcement-learning based chess-playing program. It's inspired by alphazero, 
but it's more just a tool for me to learn a bit about reinforcement learning than anything else. In other words: Don't take it too seriously.
    
The program is built in multiple layers.   

* The first layer is the basic game mechanics of chess: Move around on the board, find legal moves, print out board positions and moves etc. Chess is implemented, but there are some elements missing. In particular there is no rooking, no en passant, and no draws forced from repeated moves.   These can be added, but I felt that the subset of chess currently implemented is fine as a "minimum viable chess", for the purposes of this project. 

* The second layer is about gameplay.  How to play against an opponent, how to run tournaments and score the performance of the participants in the tournaments.   At this time the only tournaments are between two players, but they do play a number of matches 

* The third layer is about reinforcement learning.  In particular deep reinforcement learning.    It is by far the least mature of the three layers, but it is complete to the extent that it is possible to put run tournaments between players that are defined by artificual neural networks implemented using Flux.jl.

This notebook will  present each of these layers and how to access them as a developer.   The first thing to do is to include the "chez.jl" file.  This will load all the necessary libraries, and run some unit tests etc.

Some of these unit tests involve playing a short turnament, so don't be alarmed if it says so.

    
    

In [26]:
include("chez.jl")

LoadError: LoadError: invalid redefinition of constant ChessBoard
in expression starting at /Users/rmz/git/AI/chezjulia/chessboard.jl:8

## Chess pieces



With that in place, let's look at the chess pieces and the chess board.

There is a type for every color and every type of piece (rook, pawn, etc.)  It all comes together in the struct ChessPiece found in the pieces.jl file.

     struct ChessPiece
       color:: Color
       piecetype:: PieceType
       printrep:: String
       unicode:: String
     end

The "printrep" is a single character written description of the piece. Black pieces are given upper case names (like "P" for black pawns).   The "unicode" is a unicode chess symbol character, so for a black pawn it would be "♟".

All the pieces are also given symbolic names, so it's possible to refer to individual pieces by the variable that represents them:



    bp  = ChessPiece(black, pawn,   "P", "♟");
    br  = ChessPiece(black, rook,   "R", "♜");
    bk  = ChessPiece(black, knight, "G", "♞")
    bb  = ChessPiece(black, bishop, "B", "♝");
    bq  = ChessPiece(black, queen,  "Q", "♛");
    bki = ChessPiece(black, king,   "K", "♚");

    wp  = ChessPiece(white, pawn,   "p", "♙");
    wr  = ChessPiece(white, rook,   "r", "♖");
    wk  = ChessPiece(white, knight, "g", "♘");
    wb  = ChessPiece(white, bishop, "b", "♗");
    wq  = ChessPiece(white, queen,  "q", "♕");
    wki = ChessPiece(white, king,   "k", "♔");

    bs = ChessPiece(transparent, blank,  " ",  " ");
    
That's pretty much it


In [6]:
 # a black pawn will print as a string with a capital P.
bp 

"P"

## Chessboard

Iin the file "chessboard.jl" we introduce a data structure  # a black pawn will print as a string with a capital P. for holding chessboards and game positions. It's implemented as an array of chess pieces. 


We then produce a constant we call startingBoard, which contains the fully populated chessboard, as it appears in standard chess before the first move is performed.

There are some structures for describing coordinates and positioning of pieces, but not much more in chessboard.jl.


# Constructing an initial chessboard

    startingBoardArray = [
      wr wk wb wq wki wb wk wr;
      wp wp wp wp wp  wp wp wp;
      bs bs bs bs bs  bs bs bs;
      bs bs bs bs bs  bs bs bs;
      bs bs bs bs bs  bs bs bs;
      bs bs bs bs bs  bs bs bs;
      bp bp bp bp bp  bp bp bp;
      br bk bb bq bki bb bk br;
    ];

    startingBoard = ChessBoard(startingBoardArray)
    
    
 The startingBoardArray is simply an of chess pieces, and we print it using the string representation of the pieces.


In [10]:
startingBoardArray


8×8 Array{ChessPiece,2}:
 "r"  "g"  "b"  "q"  "k"  "b"  "g"  "r"
 "p"  "p"  "p"  "p"  "p"  "p"  "p"  "p"
 " "  " "  " "  " "  " "  " "  " "  " "
 " "  " "  " "  " "  " "  " "  " "  " "
 " "  " "  " "  " "  " "  " "  " "  " "
 " "  " "  " "  " "  " "  " "  " "  " "
 "P"  "P"  "P"  "P"  "P"  "P"  "P"  "P"
 "R"  "G"  "B"  "Q"  "K"  "B"  "G"  "R"

The startingBoard object however, we chose to print using the unicode "chess" characters.  This is what we will ordinarily use, since it (according to my taste) looks nicer and more compact.

In [9]:
startingBoard

8♜♞♝♛♚♝♞♜8
7♟♟♟♟♟♟♟♟7
6        6
5        5
4        4
3        3
2♙♙♙♙♙♙♙♙2
1♖♘♗♕♔♗♘♖1


## Moving around


The central data structure for representing moves is the Move sgtruct

    struct Move
        start:: Coord
        destination:: Coord
        capture:: Bool
        startPiece:: ChessPiece
        destinationPiece:: ChessPiece
    end

It's designed to be comprehensive and simple to work with, not necessarily small or computationally fast.  It describes how a piece is moved rom one place to another.  It also contains references to the piece at the source and the destination.  This information is obviously also stored on the chessboard.  The reason for the duplication is that some operations should be able to do without referencing to the board. One of those operations is simply printing the move.

There are methods  in movement.jl for printing out moves in standard algebraic notation (SAN) and figurine algebraic notation [wikipedia](https://en.wikipedia.org/wiki/Chess_notation) (move_as_san, move_as_san).  These are useful when printing games, and it's useful to have a compact notation rather than the whole chessboard (although sometimes both in combination is even more useful).

There are a few little quirks about how legal moves are generated.  They should be simple, if you are interested read the source code. For the most part, the only part that external users of this package needs to interface with is the function 

      get_moves(color::Color, board::ChessBoard, drop_king_moves::Bool  = false)

This function will assume that "color" is about to make the next move, and will generate a list of all the available moves for that color.   The drop_king_moves parameter is by default set to false, but in the case where a king-move is checked for legality, it is useful to generate all possible moves by pieces that are not kings, so hence the parameter.

Internally the get_moves function will simply find all pieces of the current color and apply the function get_mvoes_for_piece.

     function get_moves_for_piece(
           piece::{Pawn, Rook, Queen, King, ...}, 
           color::Color,
           board::ChessBoard,
           coord::Coord,
           drop_king_moves::Bool)

This is a heavily overloaded function.  There is one for each of the types of pieces (pawn, rook, ...), so essentially each type of piece has its own function that will give all the possible moves.  There are some helper functions for that too, in essence making up a small domain specific language for talking generating chess moves, but I will not say more about that here.


In [15]:
# To get all the opening moves for white do:

legal_opening_moves = get_moves(white, startingBoard)

# (Note that the elements in the 20-element array are not strings, they are Move records, they just print
# as strings.)

20-element Array{Any,1}:
 "♘b1c3"
 "♘b1a3"
 "♘g1h3"
 "♘g1f3"
 "♙a2a3"
 "♙a2a4"
 "♙b2b3"
 "♙b2b4"
 "♙c2c3"
 "♙c2c4"
 "♙d2d3"
 "♙d2d4"
 "♙e2e3"
 "♙e2e4"
 "♙f2f3"
 "♙f2f4"
 "♙g2g3"
 "♙g2g4"
 "♙h2h3"
 "♙h2h4"

If we want to apply one of these moves to the starting board, we invoke the apply_move! function:

In [17]:
apply_move!(legal_opening_moves[1], startingBoard)

8♜♞♝♛♚♝♞♜8
7♟♟♟♟♟♟♟♟7
6        6
5        5
4        4
3  ♘     3
2♙♙♙♙♙♙♙♙2
1♖ ♗♕♔♗♘♖1


... and that's pretty much all you need to abstract away most of chess's game mechanics.

## Gameplay
So we have pieces, and we have boards, but we don't yet have players, games and tournaments.  The "gameplay.jl" file introduces that.  Players are structures:

   struct Player
       id::String
       strategy
       state
   end

The id is a string to help us keep track of the players.  The strategy is a function that will take a few parameters..

     strategy(board, available_moves, move_history, board_history)

and return one of the moves listed in "available_moves".   The strategy may mutate the state of the player it's associated with, but nothing else (that's a convention, and anyone who write players are required to follow that contract, but it's not enforced by typesystem or other formal constraint-imposing mechanisms).

Game outecomes may be wins, or draws.  The wins contains a reference to the winning player.

gameplay.jl also introduces the concept of a "tournament".   A tournament is a sequence of individual games that are played between two players.  The players are alternatively set to play white and black.  The tournament simply plays the games to completion (draw due to being too long, or one of the players wins).

To facilitate unit testing a type of  player that selects random legal moves is introduced, and a simple tournament is played between two of these "random players".


In [21]:
# Get the result of a short tournament (10 games) between two random players.
random_result = play_tournament(random_player_1, random_player_2)

"Tournament_Result{games= {10}, p1='random player 1', p2='random player 2',  p1wins = 4, p2wins = 6, draws = 0}"

In [24]:
# There is a convenience function that will calculate the win-rate for the p2 player
p2_win_ratio(random_result)
# This function is later used to determine when the p2 player is significantly better than
# the p1 player.   The result shouldn't be trusted for short tournaments (due to math :-).

0.6

The function tournament_learning will run a tournament and run a learning algorithm that will attempt to select better moves after every training session.    The tournament will hopefully produce a sequence of increasingly better playes that will all be placed in the "p2" position.  In the end a dataframe containing a log some of the results found during the tourament is returned.  The player that occupies the "p2" role at the end of the tournament, presumably the best player the tournament produced, is also returned.  While running the tournament will print some progress information so that we will not think it is just sitting there doing nothing.


In [37]:
    (log, winning_player) = tournament_learning(
        10,  # Number of tournaments (10 is a very low number)
        0.55, # The trigger value for cloning the learning player
        200,  # How many rounds to play before calling the game a draw
        12)   # How many games in the tournament (12 is a very small number, but this is just a demo)

Playing tournament round 1 
round 1 p2_advantage = 0.8333333333333334
   p2( Initial q player, learning) has a 0.8333333333333334 advantage, so cloning it into p1, replacing (random player 1, static)
Q-learning
   Training: ............
Playing tournament round 2 
round 2 p2_advantage = 0.3333333333333333
Q-learning
   Training: ............
Playing tournament round 3 
round 3 p2_advantage = 0.16666666666666666
Q-learning
   Training: ............
Playing tournament round 4 
round 4 p2_advantage = 0.5
Q-learning
   Training: ............
Playing tournament round 5 
round 5 p2_advantage = 0.5
Q-learning
   Training: ............
Playing tournament round 6 
round 6 p2_advantage = 0.5833333333333334
   p2( Initial q player, learning) has a 0.5833333333333334 advantage, so cloning it into p1, replacing (Clone gen 2 q-player, static)
Q-learning
   Training: ............
Playing tournament round 7 
round 7 p2_advantage = 0.4166666666666667
Q-learning
   Training: ............
Playing tournam

(10×9 DataFrame. Omitted printing of 4 columns
│ Row │ round │ p1name               │ p2name           │ p1wins │ p2wins │
│     │ Int64 │ String               │ String           │ Int64  │ Int64  │
├─────┼───────┼──────────────────────┼──────────────────┼────────┼────────┤
│ 1   │ 1     │ random player 1      │ Initial q player │ 1      │ 10     │
│ 2   │ 2     │ Clone gen 2 q-player │ Initial q player │ 6      │ 4      │
│ 3   │ 3     │ Clone gen 2 q-player │ Initial q player │ 6      │ 2      │
│ 4   │ 4     │ Clone gen 2 q-player │ Initial q player │ 6      │ 6      │
│ 5   │ 5     │ Clone gen 2 q-player │ Initial q player │ 5      │ 6      │
│ 6   │ 6     │ Clone gen 2 q-player │ Initial q player │ 4      │ 7      │
│ 7   │ 7     │ Clone gen 3 q-player │ Initial q player │ 5      │ 5      │
│ 8   │ 8     │ Clone gen 3 q-player │ Initial q player │ 6      │ 5      │
│ 9   │ 9     │ Clone gen 3 q-player │ Initial q player │ 4      │ 8      │
│ 10  │ 10    │ Clone gen 4 q-player │ In

In [39]:
DataFrames.describe(log)

,variable,mean,min,median,max,nunique
,Symbol,Union…,Any,Union…,Any,Union…
1,round,5.5,1,5.5,10,
2,p1name,,Clone gen 2 q-player,,random player 1,4
3,p2name,,Initial q player,,Initial q player,1
4,p1wins,4.9,1,5.5,6,
5,p2wins,5.9,2,6.0,10,
6,draws,1.2,0,1.0,4,
7,p2advantage,0.491667,0.166667,0.5,0.833333,
8,cloning_triggered,0.3,0,0.0,1,
9,clone_generation,2.4,1,2.0,4,


## Future work

Actually, most of the work is probably future work.   The basic mechanics of "tournament learning" is probably in place now.  However, it's not done.   Also it would be nice to able to picle the players so that they can be used in different settings later. In particular it could be nice to play against it to see if it is any way behaving as a reasonable opponent.

Currently the network learning takes most of the time (as it should). However, it could be speeded up significantly by using GPUs.  That should be easy, but since I haven't run this code on a GPU-enabled machine yet, that part hasn't been done.  It should be a ten line (or less) code patch.

That however is not the most important part: The actual reinforcement learning algorithm, and the neural network that does the learning is really very bad.  It is little more than placeholders that has been implemented to build the mechanics necessary to start development of players in ernest. ... That's where we are now.  The way forward should be done something like this:

* Read throught the alphazero/alphago papers one more time.
* Read through some of the software that has been published that are inspired by alphazero/alphago.
* Modify the learning algorithm to be more in line with the techniques described in those papers.   
* Have discussions with people about what a reasonable implementation can be, or even better, how to construct an experiment that will give answers about that question.
* Abstract the chess out of the picture .-). By that I mean that there are a number of "layering violations" in the current software.   Knowledge about chess mechanics permeates all of the code.  However, it should be possible to make abstract games of which chess is only one instance.   That level of abstraction isn't done yet, but at some point in the not too distant future it should be done.
* Make the julia code nicer.  Add more ideomatic features, make it clearer, more consistent (camel/snake-casing for isntance).
* Make it accessible for others to tinker with.  Learning is the whole point of this project, and sharing that joy is a big part of the motivation behind doing this.